In [1]:
import pandas as pd

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
df = pd.read_csv('../data/data.csv')
documents = df.to_dict(orient='records')

In [14]:
prompt_template = """
You emulate a user of our law assistant application.
Formulate 5 questions this user might ask based on a provided content of law.
Make the questions specific to this law.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. Write in russian 

The record:

глава: {глава}
содержание: {содержание}
статья: {статья}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [15]:
prompt = prompt_template.format(**documents[0])

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
questions = llm(prompt)

In [18]:
import json

In [19]:
json.loads(questions)

{'questions': ['Какое законодательство об административных правонарушениях существует в Республике Казахстан?',
  'На каком основании основывается Кодекс об административных правонарушениях Республики Казахстан?',
  'Каким образом вносятся изменения и дополнения в Кодекс об административных правонарушениях?',
  'Какое значение имеют международные договоры для законодательства об административных правонарушениях в Казахстане?',
  'Что происходит в случае конфликта между нормами международного договора и Кодекса об административных правонарушениях?']}

In [20]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [21]:
from tqdm.auto import tqdm

In [22]:
results = {}

In [23]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/1063 [00:00<?, ?it/s]

JSONDecodeError: Expecting ',' delimiter: line 1 column 416 (char 415)

In [24]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [26]:
final_results[0]

(0,
 'Каково содержание законодательства Республики Казахстан об административных правонарушениях?')

In [27]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [28]:
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False)

In [29]:
!head ../data/ground-truth-retrieval.csv

id,question
0,Каково содержание законодательства Республики Казахстан об административных правонарушениях?
0,На каких документах основывается Кодекс об административных правонарушениях Республики Казахстан?
0,Как осуществляется внесение изменений в настоящий Кодекс?
0,Каков приоритет международных договоров по сравнению с Кодексом об административных правонарушениях?
0,Что представляет собой составная часть законодательства об административных правонарушениях?
1,Что является основанием административной ответственности согласно статье 2?
1,Какое деяние считается правонарушением в контексте административной ответственности?
1,Какие признаки должны присутствовать для признания деяния правонарушением?
1,Ссылка на какую часть Кодекса необходима для определения административной ответственности?
